# Импорт библиотек

In [ ]:
!pip install comet-ml

import comet_ml
from comet_ml import Experiment

import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install pytorch_lightning==1.9.4

In [ ]:
import pytorch_lightning as pl

logger = pl.loggers.comet.CometLogger(
    #secret logs :)
)

In [ ]:
!pip install nemo_toolkit[all]

In [ ]:
import nemo
import nemo.collections.asr as nemo_asr

[NeMo W 2023-04-17 06:52:55 optimizers:54] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2023-04-17 06:53:00 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.


In [ ]:
BRANCH = 'r1.0.0b3'

!pip install pydub
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install unidecode
!pip install omegaconf
!pip install torchaudio>=0.6.0 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
import os
import json
from google.colab import drive
from omegaconf import OmegaConf

import wave
import torch
import librosa
import torchaudio
import librosa.display
import soundfile as sf
from scipy import signal
import IPython.display as ipd
from pydub import AudioSegment
import torchaudio.functional as F 
from torchaudio.utils import download_asset

import numpy as np
import pandas as pd

import shutil
from google.colab import files

# Загрузка файлов

In [ ]:
drive.mount("/content/gdrive/")

Mounted at /content/gdrive/


In [ ]:
!mkdir '/content/dataset'
!mkdir '/content/dataset/test'
!mkdir '/content/dataset/far'

In [ ]:
!unzip '/content/gdrive/MyDrive/Colab Notebooks/neymark/farph.zip' -d '/content/dataset/far'

In [ ]:
!unzip '/content/gdrive/MyDrive/Colab Notebooks/neymark/train.zip' -d '/content/dataset/'
!unzip '/content/gdrive/MyDrive/Colab Notebooks/neymark/dev.zip' -d '/content/dataset/'
!tar -xvf '/content/gdrive/MyDrive/Colab Notebooks/neymark/v0.02_test.tar' -C '/content/dataset/test'

# Создание .json файлов для хранения путей к аудиозаписям

In [ ]:
import wave

def extract_paths(file_path, outpath):
    labels = ['marvin', 'unknow']
    out = []

    for root, dirs, files in os.walk(file_path):
        for i in dirs:
            for root2, dirs2, files2 in os.walk(file_path+i):
                for j in files2:
                    path = file_path + i + '/' + j

                    duration = 0.0
                    with wave.open(path) as mywav:
                            duration = mywav.getnframes() / mywav.getframerate()

                    out.append({"audio_filepath": path, "duration": duration, 
                                "command": 'unknow'})


    with open(outpath, 'w') as f:
        for val in out:
            json.dump(val, f)
            f.write("\n")
            f.flush()

In [ ]:
extract_paths('/content/dataset/dev/', '/content/dev.json')
extract_paths('/content/dataset/test/', '/content/test.json')
extract_paths('/content/dataset/train/', '/content/train.json')

extract_paths('/content/dataset/far/', '/content/far.json')

In [ ]:
!mkdir jsons_files
!mv dev.json jsons_files
!mv test.json jsons_files
!mv train.json jsons_files

In [ ]:
shutil.make_archive("/content/jsons_files", "zip", "/content/jsons_files")
!cp /content/jsons_files.zip "/content/gdrive/MyDrive/Colab Notebooks/neymark"
files.download('/content/jsons_files.zip')

# Скачиваем yaml файл с гитхаба и меняем его под наш датасет

In [ ]:
MODEL_CONFIG = "matchboxnet_3x1x64_v2.yaml"

if not os.path.exists(f"configs/{MODEL_CONFIG}"):
  !wget -P configs/ "https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/asr/conf/{MODEL_CONFIG}"

# Создаем переменные, хранящие пути к файлам

In [ ]:
train_dataset = os.path.join('/content/train.json')
val_dataset = os.path.join('/content/dev.json')
test_dataset = os.path.join('/content/test.json')

In [ ]:
far_dataset = os.path.join('/content/far.json')

In [ ]:
!head -n 5 {train_dataset}

{"audio_filepath": "/content/dataset/train/off/72242187_nohash_3.wav", "duration": 1.0, "command": "unknow"}
{"audio_filepath": "/content/dataset/train/off/82e7204d_nohash_2.wav", "duration": 1.0, "command": "unknow"}
{"audio_filepath": "/content/dataset/train/off/f7879738_nohash_0.wav", "duration": 1.0, "command": "unknow"}
{"audio_filepath": "/content/dataset/train/off/de89e2ca_nohash_5.wav", "duration": 1.0, "command": "unknow"}
{"audio_filepath": "/content/dataset/train/off/25132942_nohash_2.wav", "duration": 1.0, "command": "unknow"}


# Конфигурация модели

In [ ]:
config_path = f"{MODEL_CONFIG}"
config = OmegaConf.load(config_path)
print(OmegaConf.to_yaml(config))

name: MatchboxNet-3x1x64-v2
model:
  sample_rate: 44100
  timesteps: 128
  repeat: 3
  dropout: 0.2
  kernel_size_factor: 1.0
  labels_full:
  - marvin
  - unknow
  labels: ${model.labels_full}
  train_ds:
    manifest_filepath: ???
    sample_rate: ${model.sample_rate}
    labels: ${model.labels}
    batch_size: 128
    shuffle: true
  validation_ds:
    manifest_filepath: ???
    sample_rate: ${model.sample_rate}
    labels: ${model.labels}
    batch_size: 128
    shuffle: false
    val_loss_idx: 0
  test_ds:
    manifest_filepath: null
    sample_rate: ${model.sample_rate}
    labels: ${model.labels}
    batch_size: 128
    shuffle: false
    test_loss_idx: 0
  preprocessor:
    cls: nemo.collections.asr.modules.AudioToMFCCPreprocessor
    params:
      window_size: 0.025
      window_stride: 0.01
      window: hann
      n_mels: 64
      n_mfcc: 64
      n_fft: 512
  spec_augment:
    cls: nemo.collections.asr.modules.SpectrogramAugmentation
    params:
      freq_masks: 2
      ti

In [ ]:
labels = config.model.labels
sample_rate = 44100

In [ ]:
config.model.train_ds.manifest_filepath = train_dataset
config.model.validation_ds.manifest_filepath = val_dataset
config.model.test_ds.manifest_filepath = test_dataset

# Подготовка к обучению

In [ ]:
cuda = 1 if torch.cuda.is_available() else 0

config.trainer.max_epochs = 25
config.trainer.accelerator = 'gpu'

In [ ]:
config.trainer

{'gpus': 0, 'max_epochs': 25, 'max_steps': -1, 'num_nodes': 1, 'accelerator': 'gpu', 'accumulate_grad_batches': 1, 'log_every_n_steps': 1, 'val_check_interval': 0.25}

In [ ]:
conf = {'max_epochs': 25, 'num_nodes': 1, 'accelerator': 'gpu', 
        'accumulate_grad_batches': 1, 
        'log_every_n_steps': 1, 'val_check_interval': 0.25, 'logger': logger}

In [ ]:
trainer = pl.Trainer(**conf)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
asr_model = nemo_asr.models.EncDecClassificationModel(cfg=config.model, trainer=trainer)

# Обучение

In [ ]:
try:
  from google import colab
  COLAB_ENV = True
except (ImportError, ModuleNotFoundError):
  COLAB_ENV = False

In [ ]:
# Load the TensorBoard notebook extension
if COLAB_ENV:
  %load_ext tensorboard
else:
  print("To use tensorboard, please use this notebook in a Google Colab environment.")

In [ ]:
trainer.fit(asr_model)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[NeMo I 2023-04-10 14:42:45 modelPT:722] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.95, 0.5]
        eps: 1e-08
        grad_averaging: False
        lr: 0.05
        weight_decay: 0.001
    )
[NeMo I 2023-04-10 14:42:45 lr_scheduler:910] Scheduler "<nemo.core.optim.lr_scheduler.PolynomialHoldDecayAnnealing object at 0x7fd22044bb80>" 
    will be used during training (effective maximum steps = 32625) - 
    Parameters : 
    (power: 2.0
    warmup_ratio: 0.05
    hold_ratio: 0.45
    min_lr: 0.001
    last_epoch: -1
    max_steps: 32625
    )


INFO:pytorch_lightning.callbacks.model_summary:
  | Name              | Type                             | Params
-----------------------------------------------------------------------
0 | spec_augmentation | SpectrogramAugmentation          | 0     
1 | crop_or_pad       | CropOrPadSpectrogramAugmentation | 0     
2 | preprocessor      | AudioToMFCCPreprocessor          | 0     
3 | encoder           | ConvASREncoder                   | 104 K 
4 | decoder           | ConvASRDecoderClassification     | 258   
5 | loss              | CrossEntropyLoss                 | 0     
6 | _accuracy         | TopKClassificationAccuracy       | 0     
-----------------------------------------------------------------------
104 K     Trainable params
0         Non-trainable params
104 K     Total params
0.419     Total estimated model params size (MB)
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are 

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=25` reached.
COMET WARNING: Couldn't retrieve Google Colab notebook content
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml ExistingExperiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/annanet/asr-keyword-spotting/392209cfcd0647e6a9262c0ef36ccd97
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     global_step [32625]                   : (0.0, 32624.0)
COMET INFO:     learning_rate [32625]                 : (3.0637256713816896e-05, 0.05000000074505806)
COMET INFO:     loss [3263]                           : (0.012741316109895706, 0.7475473284721375)
COMET INFO:     train_loss [32625]                    : (0.007426457945257425, 1.0545167922973633)

## Смотрим на тестовую выборку

In [ ]:
trainer.test(asr_model, ckpt_path=None)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/annanet/asr-keyword-spotting/392209cfcd0647e6a9262c0ef36ccd97



Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│     test_epoch_top@1      │    0.9932515621185303     │
│         test_loss         │   0.043291617184877396    │
└───────────────────────────┴───────────────────────────┘

COMET WARNING: Couldn't retrieve Google Colab notebook content
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml ExistingExperiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/annanet/asr-keyword-spotting/392209cfcd0647e6a9262c0ef36ccd97
COMET INFO:   Metrics:
COMET INFO:     test_epoch_top@1 : 0.9932515621185303
COMET INFO:     test_loss        : 0.043291617184877396
COMET INFO:   Others:
COMET INFO:     Created from : pytorch-lightning
COMET INFO:     Name         : augm_train_dev+44100
COMET INFO:   Parameters:
COMET INFO:     cfg/crop_or_pad_augment/_target_     : nemo.collections.asr.modules.CropOrPadSpectrogramAugmentation
COMET INFO:     cfg/crop_or_pad_augment/audio_length : 128
COMET INFO:     cfg/decoder/_target_         

[{'test_loss': 0.043291617184877396, 'test_epoch_top@1': 0.9932515621185303}]

# Результаты train, dev, test

In [ ]:
config.model.test_ds.sample_rate

44100

In [ ]:
restored_model = nemo_asr.models.EncDecClassificationModel.load_from_checkpoint('/content/epoch=24-step=32624.ckpt')

restored_model.setup_multiple_test_data({'manifest_filepath': os.path.join('/content/test.json'), 'sample_rate': 44100, 'labels': ['marvin', 'unknow'], 'batch_size': 128, 'shuffle': False})
restored_model.setup_multiple_validation_data({'manifest_filepath': os.path.join('/content/dev.json'), 'sample_rate': 44100, 'labels': ['marvin', 'unknow'], 'batch_size': 128, 'shuffle': False})
restored_model.setup_training_data({'manifest_filepath': os.path.join('/content/train.json'), 'sample_rate': 44100, 'labels': ['marvin', 'unknow'], 'batch_size': 128, 'shuffle': False})

[NeMo I 2023-04-17 10:25:42 collections:298] Filtered duration for loading collection is 0.000000.
[NeMo I 2023-04-17 10:25:42 collections:301] Dataset loaded with 4890 items, total duration of  1.36 hours.
[NeMo I 2023-04-17 10:25:42 collections:303] # 4890 files loaded accounting to # 2 labels
[NeMo I 2023-04-17 10:25:43 collections:298] Filtered duration for loading collection is 0.000000.
[NeMo I 2023-04-17 10:25:43 collections:301] Dataset loaded with 19692 items, total duration of  5.40 hours.
[NeMo I 2023-04-17 10:25:43 collections:303] # 19692 files loaded accounting to # 2 labels
[NeMo I 2023-04-17 10:25:53 collections:298] Filtered duration for loading collection is 0.000000.
[NeMo I 2023-04-17 10:25:53 collections:301] Dataset loaded with 166930 items, total duration of  46.19 hours.
[NeMo I 2023-04-17 10:25:53 collections:303] # 166930 files loaded accounting to # 2 labels


In [ ]:
test_dl = restored_model._test_dl
dev_dl = restored_model._validation_dl
train_dl = restored_model._train_dl

In [ ]:
cpu_model = restored_model.cpu()
cpu_model.eval()

In [ ]:
@torch.no_grad()
def extract_logits(model, dataloader):
  logits_buffer = []
  label_buffer = []

  # Follow the above definition of the test_step
  for batch in dataloader:
    audio_signal, audio_signal_len, labels, labels_len = batch
    logits = model(input_signal=audio_signal, input_signal_length=audio_signal_len)

    logits_buffer.append(logits)
    label_buffer.append(labels)
    print(".", end='')
  print()
  
  print("Finished extracting logits !")
  logits = torch.cat(logits_buffer, 0)
  labels = torch.cat(label_buffer, 0)
  return logits, labels


In [ ]:
def get_labels_logits(model, dl):
    logits, labels = extract_logits(model, dl)
    print("Logits:", logits.shape, "Labels :", labels.shape)

    return logits, labels

In [ ]:
clogits_train, labels_train = get_labels_logits(cpu_model, train_dl)

logits_test, labels_test = get_labels_logits(cpu_model, test_dl)

logits_dev, labels_dev = get_labels_logits(cpu_model, dev_dl)

acc = cpu_model._accuracy(logits=logits_train, labels=labels_train)
print("Accuracy : ", float(acc[0]*100))

acc = cpu_model._accuracy(logits=logits_dev, labels=labels_dev)
print("Accuracy : ", float(acc[0]*100))

acc = cpu_model._accuracy(logits=logits_test, labels=labels_test)
print("Accuracy : ", float(acc[0]*100))

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

## Метрики

In [ ]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score

In [ ]:
probs_test = torch.softmax(logits_test, dim=-1)
probas_test, preds_test = torch.max(probs_test, dim=-1)

print(balanced_accuracy_score(labels_test, preds_test))

probs_train = torch.softmax(logits_train, dim=-1)
probas_train, preds_train = torch.max(probs_train, dim=-1)

print(balanced_accuracy_score(labels_train, preds_train))

probs_dev = torch.softmax(logits_dev, dim=-1)
probas_dev, preds_dev = torch.max(probs_dev, dim=-1)

print(balanced_accuracy_score(labels_dev, preds_dev))

print(f1_score(labels_test, preds_test))
print(f1_score(labels_train, preds_train))
print(f1_score(labels_dev, preds_dev))

0.5434049451266094
0.9866950218654525
0.9818708104814138
0.9969224456298729
0.9868195386541805
0.9821204988230581


# Посмотрим на неверно классифицированные данные

In [ ]:
# для вывода labels
class ReverseMapLabel:
    def __init__(self, data_loader):
        self.label2id = dict(data_loader.dataset.label2id)
        self.id2label = dict(data_loader.dataset.id2label)

    def __call__(self, pred_idx, label_idx):
        return self.id2label[pred_idx], self.id2label[label_idx]

In [ ]:
def indices_inc_lables(dl, logits, labels, cpu_model):
    sample_idx = 0
    incorrect_preds = []
    rev_map = ReverseMapLabel(dl)

    probs = torch.softmax(logits, dim=-1)
    probas, preds = torch.max(probs, dim=-1)

    total_count = cpu_model._accuracy.total_counts_k[0]
    incorrect_ids = (preds != labels).nonzero()
    for idx in incorrect_ids:
        proba = float(probas[idx][0])
        pred = int(preds[idx][0])
        label = int(labels[idx][0])
        idx = int(idx[0]) + sample_idx

        incorrect_preds.append((idx, *rev_map(pred, label), proba))

    print(f"Num test samples : {total_count.item()}")
    print(f"Num errors : {len(incorrect_preds)}")

    incorrect_preds = sorted(incorrect_preds, key=lambda x: x[-1], reverse=False)

    return incorrect_preds

In [ ]:
# train тут напутано с Num test samples, должно быть 166930
incorrect_preds_train = indices_inc_lables(train_dl, logits_train, labels_train, cpu_model)

Num test samples : 7198
Num errors : 2221


In [ ]:
#dev тут напутано с Num test samples, должно быть 19692
incorrect_preds_dev = indices_inc_lables(dev_dl, logits_dev, labels_dev, cpu_model)

Num test samples : 7198
Num errors : 357


In [ ]:
#test
incorrect_preds_test = indices_inc_lables(test_dl, logits_test, labels_test, cpu_model)

Num test samples : 4890
Num errors : 30


In [ ]:
for incorrect_sample in incorrect_preds_test:
    print(str(incorrect_sample))

(204, 'marvin', 'unknow', 0.5164039134979248)
(88, 'marvin', 'unknow', 0.5786099433898926)
(1182, 'marvin', 'unknow', 0.5983117818832397)
(2057, 'marvin', 'unknow', 0.608277440071106)
(2191, 'marvin', 'unknow', 0.6127679944038391)
(192, 'marvin', 'unknow', 0.6413969397544861)
(4811, 'marvin', 'unknow', 0.6846175789833069)
(3276, 'marvin', 'unknow', 0.7043545246124268)
(1250, 'marvin', 'unknow', 0.7333923578262329)
(270, 'marvin', 'unknow', 0.7750107049942017)
(3897, 'unknow', 'marvin', 0.8184032440185547)
(3902, 'unknow', 'marvin', 0.8411893844604492)
(2323, 'marvin', 'unknow', 0.8528593182563782)
(2568, 'marvin', 'unknow', 0.8629851937294006)
(3903, 'unknow', 'marvin', 0.8660202622413635)
(3898, 'unknow', 'marvin', 0.8812824487686157)
(3901, 'unknow', 'marvin', 0.8847242593765259)
(3904, 'unknow', 'marvin', 0.894956111907959)
(3895, 'unknow', 'marvin', 0.8972187042236328)
(2409, 'marvin', 'unknow', 0.9129598140716553)
(3900, 'unknow', 'marvin', 0.9274628162384033)
(3896, 'unknow', 'ma

# FAR per hour

In [ ]:
restored_model.setup_multiple_test_data({'manifest_filepath': os.path.join('/content/far.json'), 'sample_rate': 44100, 'labels': ['marvin', 'unknow'], 'batch_size': 128, 'shuffle': False})
far_dl = restored_model._test_dl

[NeMo I 2023-04-17 10:29:26 collections:298] Filtered duration for loading collection is 0.000000.
[NeMo I 2023-04-17 10:29:26 collections:301] Dataset loaded with 7198 items, total duration of  2.00 hours.
[NeMo I 2023-04-17 10:29:26 collections:303] # 7198 files loaded accounting to # 1 labels


In [ ]:
cpu_model = restored_model.cpu()
cpu_model.eval()
logits_far, labels_far = extract_logits(cpu_model, far_dl)
print("Logits:", logits_far.shape, "Labels :", labels_far.shape)

acc = cpu_model._accuracy(logits=logits_far, labels=labels_far)
print("Accuracy : ", float(acc[0]*100))

.........................................................
Finished extracting logits !
Logits: torch.Size([7198, 2]) Labels : torch.Size([7198])
Accuracy :  100.0


In [ ]:
#far
incorrect_preds_far = indices_inc_lables(far_dl, logits_far, labels_far, cpu_model)

Num test samples : 7198
Num errors : 0
